In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

In [2]:
#constants
board_size = 5
board_dim = 3 #convey 3 pieces of info: halite, shipyards, ships
agent_count = 4
max_cell_halite = 9

#create env and get first state
environment = make("halite", configuration={"size": board_size, "startingHalite": 1000})
environment.reset(agent_count)
state = environment.state[0]

In [3]:
'''
Board is nxn where n is board_size
Number represents cell halite
Lower case = ship
Upper case = shipyard
a,b,c,etc. = player 1, player 2, player 3, etc.
'''
board = Board(state.observation, environment.configuration)
print(board)

| 0 | 0 | 0 | 0 | 0 |
| 0 |a3 | 0 |b3 | 0 |
| 0 | 0 | 0 | 0 | 0 |
| 0 |c3 | 0 |d3 | 0 |
| 0 | 0 | 0 | 0 | 0 |



In [4]:
#Encodes halite board object to np array
def board_to_np(board):
    #convert board to string and clean up
    board_np = str(board).replace(' ', "")
    board_np = board_np.replace('|\n', "")

    #tokenize board to insert into np array
    board_np = np.array((board_np.split("|"))[1:])

    #create placeholder array that will hold board
    board_arr = np.zeros((board_dim, board_size**2))
    
    #iterate through cells and fill in placeholder array
    for i in range(board_size**2):
        cell = board_np[i]
        for j in cell:
            if j>='0' and j<='9':    #using unicode to determine character
                board_arr[0,i] = int(j)/max_cell_halite    #normalize between 0-1
            elif j>='A' and j<='Z':
                board_arr[1,i] = (ord(j)-64)/agent_count    
            elif j>='a' and j<='z':
                board_arr[2,i] = (ord(j)-96)/agent_count
    board_arr = board_arr.reshape(board_dim, board_size, board_size)
    
    #return np array of size n x n x 3
    return board_arr

In [16]:
start = time.time()
board_np = board_to_np(board)
wall_time = time.time()-start
print(board_np)
print(f"Encoding the board took {wall_time} seconds")

[[[0.         0.         0.         0.         0.        ]
  [0.         0.33333333 0.         0.33333333 0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.33333333 0.         0.33333333 0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[0.         0.         0.         0.         0.        ]
  [0.         0.25       0.         0.5        0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.75       0.         1.         0.        ]
  [0.         0.         0.         0.         0.        ]]]
Encoding the board took 0.0009946823120117188 seconds


In [6]:
print(board)

| 0 | 0 | 0 | 0 | 0 |
| 0 |a3 | 0 |b3 | 0 |
| 0 | 0 | 0 | 0 | 0 |
| 0 |c3 | 0 |d3 | 0 |
| 0 | 0 | 0 | 0 | 0 |



In [24]:
#further time testing
iters = 500
times = []
for i in range(iters):
    start = time.time()
    board_np = board_to_np(board)
    times.append(time.time()-start)
print("Average time:", np.array(times).mean())

Average time: 0.00015565109252929686
